In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.nighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

print("klar")